<a href="https://colab.research.google.com/github/BhojRajBist/BhojRajBist/blob/main/NDVI_from_the_Sentinelhub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install the necessary libraries
!pip install requests requests-oauthlib

from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

# Your Sentinel Hub Client Credentials (replace with your actual values)
client_id = '0948b6f4-7a85-4786-8d91-43fecc9c67f6'
client_secret = 'kg7yFgXhGlA4u4f4XRUxfKOhEbrifunz'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(
    token_url='https://services.sentinel-hub.com/auth/realms/main/protocol/openid-connect/token',
    client_secret=client_secret,
    include_client_id=True
)

# Define the compliance hook to check for valid status codes
def sentinelhub_compliance_hook(response):
    response.raise_for_status()  # Raise an error for bad status codes
    return response

# Register the compliance hook to handle the access token response
oauth.register_compliance_hook("access_token_response", sentinelhub_compliance_hook)

# Test the authentication by making a request to the Sentinel Hub API
try:
    # Example API request (Sentinel Hub configuration endpoint)
    response = oauth.get("https://services.sentinel-hub.com/configuration/v1/wms/instances")
    print(response.json())  # Output the response from Sentinel Hub (in JSON format)
except Exception as e:
    print(f"An error occurred: {e}")


[{'@id': 'https://services.sentinel-hub.com/configuration/v1/wms/instances/4955d42f-5001-4813-948f-b3d492db8a08', 'id': '4955d42f-5001-4813-948f-b3d492db8a08', 'name': 'Sentinel 2 WMS template', 'domainAccountId': 'a0818a9a-e541-4a76-afd9-40e595d709f4', 'additionalData': {'showWarnings': True, 'showLogo': True, 'imageQuality': 90}, 'created': '2024-10-24T03:37:29.851683Z', 'lastUpdated': '2024-10-24T03:37:29.851683Z', 'layers': {'@id': 'https://services.sentinel-hub.com/configuration/v1/wms/instances/4955d42f-5001-4813-948f-b3d492db8a08/layers'}}, {'@id': 'https://services.sentinel-hub.com/configuration/v1/wms/instances/9c13e458-24fb-4c5f-9c61-b7db76481a70', 'id': '9c13e458-24fb-4c5f-9c61-b7db76481a70', 'name': 'Planet Sandbox Data template', 'domainAccountId': 'a0818a9a-e541-4a76-afd9-40e595d709f4', 'additionalData': {'showWarnings': True, 'showLogo': True, 'imageQuality': 90, 'disabled': False}, 'created': '2024-10-24T03:37:30.117812Z', 'lastUpdated': '2024-10-24T03:37:30.117812Z', '

In [3]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [
      {datasource: "s2", bands: ["B04", "B08", "dataMask"]},
      {datasource: "s1", bands: ["VV", "dataMask"]}
    ],
    output: [
      {
        id: "ndvi",
        bands: 1
      },
      {
        id: "dataMask",
        bands: 1
      }],
    mosaicking: "SIMPLE"
  };
}

function evaluatePixel(samples) {
  let ndvi = (samples.s2[0].B08 - samples.s2[0].B04) / (samples.s2[0].B08+samples.s2[0].B04);

  // Create a mask for invalid ndvi value
  let validNDVIMask = 1;
  if (!isFinite(ndvi)) {
    validNDVIMask = 0;
  }

  // Classify NDVI into categories: Water, Vegetation
  let ndvi_class = "Unknown";
  if (ndvi <= 0.2) {
    ndvi_class = "Water";
  } else if (ndvi > 0.2 && ndvi <= 0.5) {
    ndvi_class = "Bare Soil";
  } else if (ndvi > 0.5) {
    ndvi_class = "Vegetation";
  }

  return {
      ndvi: [ndvi],
      dataMask: [samples.s2[0].dataMask * samples.s1[0].dataMask * validNDVIMask],
      classification: [ndvi_class]
  };
}
"""


In [9]:
!pip install requests

import requests

In [17]:
import requests
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import json
import geopandas as gpd
from shapely.geometry import shape

# Your Sentinel Hub Client Credentials (replace with your actual values)
client_id = '0948b6f4-7a85-4786-8d91-43fecc9c67f6'
client_secret = 'kg7yFgXhGlA4u4f4XRUxfKOhEbrifunz'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(
    token_url='https://services.sentinel-hub.com/auth/realms/main/protocol/openid-connect/token',
    client_secret=client_secret,
    include_client_id=True
)

# Define the compliance hook to check for valid status codes
def sentinelhub_compliance_hook(response):
    response.raise_for_status()  # Raise an error for bad status codes
    return response

# Register the compliance hook to handle the access token response
oauth.register_compliance_hook("access_token_response", sentinelhub_compliance_hook)
# Sentinel Hub evalscript for NDVI calculation excluding water pixels
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B04",
        "B08",
        "SCL",
        "dataMask"
      ]
    }],
    output: [
      {
        id: "data",
        bands: 1
      },
      {
        id: "dataMask",
        bands: 1
      }]
  }
}

function evaluatePixel(samples) {
    let ndvi = (samples.B08 - samples.B04) / (samples.B08 + samples.B04);
    var validNDVIMask = samples.B08 + samples.B04 !== 0 ? 1 : 0;
    var noWaterMask = samples.SCL === 6 ? 0 : 1;

    return {
        data: [ndvi],
        dataMask: [samples.dataMask * validNDVIMask * noWaterMask]
    };
}
"""

# Define the request body with polygon coordinates and other parameters
stats_request = {
    "input": {
        "bounds": {
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [458085.878866, 5097236.833044],
                    [457813.834156, 5096808.351383],
                    [457979.897062, 5096313.767184],
                    [458146.639373, 5096405.411294],
                    [458085.878866, 5097236.833044]
                ]]
            },
            "properties": {
                "crs": "http://www.opengis.net/def/crs/EPSG/0/32633"
            }
        },
        "data": [
            {
                "type": "sentinel-2-l2a",
                "dataFilter": {
                    "mosaickingOrder": "leastCC"
                }
            }
        ]
    },
    "aggregation": {
        "timeRange": {
            "from": "2020-01-01T00:00:00Z",
            "to": "2020-12-31T00:00:00Z"
        },
        "aggregationInterval": {
            "of": "P30D"
        },
        "evalscript": evalscript,
        "resx": 10,
        "resy": 10
    }
}

# Sentinel Hub API endpoint
url = "https://services.sentinel-hub.com/api/v1/statistics"

# Send the request to Sentinel Hub API
response = requests.post(url, headers=headers, json=stats_request)
sh_statistics = response.json()

# Function to classify NDVI values
def classify_ndvi(ndvi_value):
    if ndvi_value < 0.2:
        return 'Water'
    elif 0.2 <= ndvi_value < 0.5:
        return 'Vegetation Sparse'
    elif 0.5 <= ndvi_value < 0.7:
        return 'Vegetation Moderate'
    else:
        return 'Vegetation Dense'

# Process and print the NDVI statistics, classify NDVI, and format as GeoJSON
classified_features = []

for interval in sh_statistics['data']:
    ndvi_stats = interval['outputs']['data']['bands']['B0']['stats']

    # Extract and classify NDVI statistics if valid values are present
    for ndvi_stat in ['min', 'max', 'mean']:
        if isinstance(ndvi_stats[ndvi_stat], (float, int)):  # Ensure numeric values
            classification = classify_ndvi(ndvi_stats[ndvi_stat])
            classified_features.append({
                "type": "Feature",
                "properties": {
                    "interval_start": interval['interval']['from'],
                    "interval_end": interval['interval']['to'],
                    "ndvi_stat": ndvi_stat,
                    "value": ndvi_stats[ndvi_stat],
                    "classification": classification
                },
                "geometry": stats_request['input']['bounds']['geometry']
            })

# Create a GeoJSON object with classified NDVI data
geojson_output = {
    "type": "FeatureCollection",
    "features": classified_features
}

#Save the classified GeoJSON to a file
with open('classified_ndvi_output.geojson', 'w') as f:
    json.dump(geojson_output, f)
# Output GeoJSON result

print(json.dumps(geojson_output, indent=2))


{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "interval_start": "2020-01-01T00:00:00Z",
        "interval_end": "2020-01-31T00:00:00Z",
        "ndvi_stat": "min",
        "value": 0.24306687712669373,
        "classification": "Vegetation Sparse"
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              458085.878866,
              5097236.833044
            ],
            [
              457813.834156,
              5096808.351383
            ],
            [
              457979.897062,
              5096313.767184
            ],
            [
              458146.639373,
              5096405.411294
            ],
            [
              458085.878866,
              5097236.833044
            ]
          ]
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "interval_start": "2020-01-01T00:00:00Z",
        "interval_end

In [33]:
import requests
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import json
import geopandas as gpd
from shapely.geometry import shape, mapping
import pyproj
from shapely.ops import transform

# Your Sentinel Hub Client Credentials (replace with your actual values)
client_id = '0948b6f4-7a85-4786-8d91-43fecc9c67f6'
client_secret = 'kg7yFgXhGlA4u4f4XRUxfKOhEbrifunz'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(
    token_url='https://services.sentinel-hub.com/auth/realms/main/protocol/openid-connect/token',
    client_secret=client_secret,
    include_client_id=True
)

# Define the compliance hook to check for valid status codes
def sentinelhub_compliance_hook(response):
    response.raise_for_status()  # Raise an error for bad status codes
    return response

# Register the compliance hook to handle the access token response
oauth.register_compliance_hook("access_token_response", sentinelhub_compliance_hook)
# Sentinel Hub evalscript for NDVI calculation excluding water pixels
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B04",
        "B08",
        "SCL",
        "dataMask"
      ]
    }],
    output: [
      {
        id: "data",
        bands: 1
      },
      {
        id: "dataMask",
        bands: 1
      }]
  }
}

function evaluatePixel(samples) {
    let ndvi = (samples.B08 - samples.B04) / (samples.B08 + samples.B04);
    var validNDVIMask = samples.B08 + samples.B04 !== 0 ? 1 : 0;
    var noWaterMask = samples.SCL === 6 ? 0 : 1;

    return {
        data: [ndvi],
        dataMask: [samples.dataMask * validNDVIMask * noWaterMask]
    };
}
"""

# Define the request body with polygon coordinates and other parameters
stats_request = {
    "input": {
        "bounds": {
            "geometry": {
                "type": "Polygon",
                "coordinates": [  [
            [
              83.91725391547527,
              28.245143588402897
            ],
            [
              83.91725391547527,
              28.18720261340046
            ],
            [
              83.99554649381724,
              28.18720261340046
            ],
            [
              83.99554649381724,
              28.245143588402897
            ],
            [
              83.91725391547527,
              28.245143588402897
            ]
          ]]
            },
            "properties": {
                "crs": "http://www.opengis.net/def/crs/EPSG/0/32633"
            }
        },
        "data": [
            {
                "type": "sentinel-2-l2a",
                "dataFilter": {
                    "mosaickingOrder": "leastCC"
                }
            }
        ]
    },
    "aggregation": {
        "timeRange": {
            "from": "2020-01-01T00:00:00Z",
            "to": "2020-12-31T00:00:00Z"
        },
        "aggregationInterval": {
            "of": "P30D"
        },
        "evalscript": evalscript,
        "resx": 10,
        "resy": 10
    }
}

# Sentinel Hub API endpoint
url = "https://services.sentinel-hub.com/api/v1/statistics"

# Send the request to Sentinel Hub API
headers = {
    'Authorization': f'Bearer {token["access_token"]}',
    'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers, json=stats_request)
sh_statistics = response.json()

# Function to classify NDVI values
def classify_ndvi(ndvi_value):
    if ndvi_value < 0.2:
        return 'Water'
    elif 0.2 <= ndvi_value < 0.5:
        return 'Vegetation Sparse'
    elif 0.5 <= ndvi_value < 0.7:
        return 'Vegetation Moderate'
    else:
        return 'Vegetation Dense'

# Reproject function from EPSG:32633 to EPSG:4326
def reproject_geometry(geometry, src_epsg="EPSG:32633", dst_epsg="EPSG:4326"):
    src_proj = pyproj.CRS(src_epsg)
    dst_proj = pyproj.CRS(dst_epsg)
    project = pyproj.Transformer.from_crs(src_proj, dst_proj, always_xy=True).transform
    return transform(project, shape(geometry))

# Process and print the NDVI statistics, classify NDVI, and format as GeoJSON
classified_features = []

for interval in sh_statistics['data']:
    ndvi_stats = interval['outputs']['data']['bands']['B0']['stats']

    # Extract and classify NDVI statistics if valid values are present
    for ndvi_stat in ['min', 'max', 'mean']:
        if isinstance(ndvi_stats[ndvi_stat], (float, int)):  # Ensure numeric values
            classification = classify_ndvi(ndvi_stats[ndvi_stat])
            # Reproject geometry to WGS84 (EPSG:4326)
            reprojected_geometry = reproject_geometry(stats_request['input']['bounds']['geometry'])
            classified_features.append({
                "type": "Feature",
                "properties": {
                    "interval_start": interval['interval']['from'],
                    "interval_end": interval['interval']['to'],
                    "ndvi_stat": ndvi_stat,
                    "value": ndvi_stats[ndvi_stat],
                    "classification": classification
                },
                "geometry": mapping(reprojected_geometry)
            })

# Create a GeoJSON object with classified NDVI data
geojson_output = {
    "type": "FeatureCollection",
    "features": classified_features
}

# Save the classified GeoJSON to a file
with open('classified_ndvi_output.geojson', 'w') as f:
    json.dump(geojson_output, f)

# Output GeoJSON result
print(json.dumps(geojson_output, indent=2))


{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "interval_start": "2020-01-01T00:00:00Z",
        "interval_end": "2020-01-31T00:00:00Z",
        "ndvi_stat": "min",
        "value": 0.7091140151023865,
        "classification": "Vegetation Dense"
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              10.512007930357713,
              0.00025475382899319384
            ],
            [
              10.512007930357893,
              0.0002542312370229577
            ],
            [
              10.51200863178157,
              0.0002542312372689043
            ],
            [
              10.51200863178139,
              0.000254753829239646
            ],
            [
              10.512007930357713,
              0.00025475382899319384
            ]
          ]
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "inter

In [34]:
import folium
import json
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon
from shapely.ops import unary_union
import geopandas as gpd

# Load the classified NDVI GeoJSON file
with open('classified_ndvi_output.geojson') as f:
    geojson_data = json.load(f)

# Calculate the centroid of the GeoJSON polygons to center the map
geometries = [shape(feature["geometry"]) for feature in geojson_data["features"]]
merged_polygon = unary_union(geometries)  # Merge all geometries to get the boundary
centroid = merged_polygon.centroid  # Calculate the centroid

# Create a map centered on the centroid of the GeoJSON geometries
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=12)

# Add the GeoJSON overlay to the map
folium.GeoJson(
    geojson_data,
    name="NDVI Classification",
    style_function=lambda feature: {
        "fillColor": "#228B22" if feature["properties"]["classification"] == "Vegetation Dense" else
                     "#32CD32" if feature["properties"]["classification"] == "Vegetation Moderate" else
                     "#ADFF2F" if feature["properties"]["classification"] == "Vegetation Sparse" else
                     "#87CEEB",
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.6,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["interval_start", "interval_end", "ndvi_stat", "value", "classification"],
        aliases=["Interval Start", "Interval End", "NDVI Stat", "Value", "Classification"]
    )
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save the map to an HTML file and display it
m.save("classified_ndvi_map.html")
m


In [36]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 45.4 MB/s eta 0:00:00


In [44]:
import requests
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import numpy as np
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape, mapping
import geopandas as gpd
import json

# Sentinel Hub Client Credentials
client_id = '0948b6f4-7a85-4786-8d91-43fecc9c67f6'
client_secret = 'kg7yFgXhGlA4u4f4XRUxfKOhEbrifunz'


# OAuth2 Authentication
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(
    token_url='https://services.sentinel-hub.com/auth/realms/main/protocol/openid-connect/token',
    client_secret=client_secret,
    include_client_id=True
)

# Define the evalscript for NDVI at pixel level
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [{ bands: ["B04", "B08", "dataMask"] }],
    output: { bands: 1, sampleType: "FLOAT32" }
  };
}

function evaluatePixel(sample) {
  let ndvi = (sample.B08 - sample.B04) / (sample.B08 + sample.B04);
  return [ ndvi ];
}
"""

# Define the request body for Process API
process_request = {
    "input": {
        "bounds": {
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [458085.878866, 5097236.833044],
                    [457813.834156, 5096808.351383],
                    [457979.897062, 5096313.767184],
                    [458146.639373, 5096405.411294],
                    [458085.878866, 5097236.833044]
                ]]
            },
            "properties": {
                "crs": "http://www.opengis.net/def/crs/EPSG/0/32633"
            }
        },
        "data": [
            {
                "type": "sentinel-2-l2a",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-01-01T00:00:00Z",
                        "to": "2020-12-31T00:00:00Z"
                    }
                }
            }
        ]
    },
    "output": {
        "width": 512,  # Adjust width for resolution
        "height": 512,  # Adjust height for resolution
        "responses": [{"identifier": "default", "format": {"type": "image/tiff"}}]
    },
    "evalscript": evalscript
}

# Sentinel Hub Process API endpoint
url = "https://services.sentinel-hub.com/api/v1/process"

# Send request to Process API
response = oauth.post(url, json=process_request)
with open('ndvi_pixel_data.tif', 'wb') as f:
    f.write(response.content)

# Read the TIFF file and extract NDVI values for each pixel
with rasterio.open('ndvi_pixel_data.tif') as src:
    image = src.read(1)  # NDVI values as a numpy array
    mask = src.read_masks(1)  # Get valid data mask
    transform = src.transform

# Convert pixel NDVI values to GeoJSON features
shapes_gen = shapes(image, mask=mask, transform=transform)
features = [
    {
        "type": "Feature",
        "geometry": mapping(shape(geom)),
        "properties": {"ndvi": value}
    }
    for geom, value in shapes_gen if value is not None
]

# Convert to GeoJSON
geojson_output = {
    "type": "FeatureCollection",
    "features": features
}

# Save GeoJSON to a file
with open("ndvi_pixel_level_output.geojson", "w") as f:
    json.dump(geojson_output, f)

# Load GeoJSON as a GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(features, crs="EPSG:32633")
gdf = gdf.to_crs(epsg=4326)  # Convert to WGS84 for accurate mapping in folium

# Initialize map centered on the AOI's centroid after reprojection
centroid = gdf.to_crs(epsg=3857).geometry.centroid.to_crs(epsg=4326)
m = folium.Map(location=[centroid.y.mean(), centroid.x.mean()], zoom_start=13)

# Add GeoJSON layer with NDVI classification styling
def style_function(feature):
    ndvi = feature["properties"]["ndvi"]
    # Handle None values for NDVI
    if ndvi is None:
        color = "grey"  # Assign grey color for None values
    else:
        color = "blue" if ndvi < 0.2 else "yellow" if ndvi < 0.5 else "green" if ndvi < 0.7 else "darkgreen"
    return {"color": color, "weight": 1, "fillOpacity": 0.6}

GeoJson(gdf.to_json(), style_function=style_function).add_to(m)

# Save map to HTML file
m.save("ndvi_pixel_level_map.html")

print("Map saved as ndvi_pixel_level_map.html")



Map saved as ndvi_pixel_level_map.html
